TRANSFORMACIONES

In [149]:
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import datetime

Funciones

In [150]:
def separar_df(df):
    filtro = df['torneo'].str.contains('Men|ATP', regex=True, case = False)
    masc = df[filtro]
    return masc

def asignar_año(fecha_str):
    # Limpiar y convertir a día y mes (sin año)
    dia_mes = fecha_str.strip('/')  # Elimina '/' final
    dia, mes = map(int, dia_mes.split('/'))
    
    # Asignar 2024 si es antes de 11 de junio, 2025 en caso contrario
    if (mes < 6) or (mes == 6 and dia <= 11):
        return f"{fecha_str}2025"
    else:
        return f"{fecha_str}2024"

def df_jugador(jugador, masc):

    filtro_jugador = (masc['jugadorA'] == jugador) | (masc['jugadorB'] == jugador)
    df_jugador = masc[filtro_jugador].copy()

    df_jugador_A = df_jugador[df_jugador['jugadorA'] == jugador].copy()
    df_jugador_B = df_jugador[df_jugador['jugadorB'] == jugador].copy()

    columnas_A = [col for col in df_jugador.columns if col.endswith('A')]
    columnas_B = [col for col in df_jugador.columns if col.endswith('B')]
    mapeo_columnas = {**{col_a: col_b for col_a, col_b in zip(columnas_A, columnas_B)},
                    **{col_b: col_a for col_a, col_b in zip(columnas_A, columnas_B)}}

    df_jugador_B_invertido = df_jugador_B.rename(columns=mapeo_columnas)

    df_final = pd.concat([df_jugador_A, df_jugador_B_invertido], ignore_index=True)

    if 'fecha' in df_final.columns:
        df_final = df_final.sort_values('fecha', ascending=False)

    return df_final


Leo los datos.

In [151]:
masc1 = pd.read_csv(".\\data\\grandslams_masc_2025-06-10.csv")
masc2 = separar_df(pd.read_csv(".\\data\\partidos_tenis_2025-06-11.csv"))
masc3 = pd.read_csv(".\\data\\lyon_perugia_masc_2025-06-11.csv")
masc4 = pd.read_csv(".\\data\\santafe_masc_2025-06-11.csv")
masc5 = pd.read_csv(".\\data\\villeneuveloubet-varnamo_masc_2025-06-11.csv")

dfs_masc = [masc1, masc2, masc3, masc4, masc5]
masc = pd.concat(dfs_masc, ignore_index=True)

masc = masc[(masc['setsA'] != 'Fils A.') & (masc['setsA'] != 'Gaston H.') & (masc['setsA'] != 'Djokovic N.') & (masc['setsA'] != 'De Minaur A.')] #elimino 4 datos mal extraídos

masc['setsA'] = masc['setsA'].astype(int)
masc['setsB'] = masc['setsB'].astype(int)

filtroGrandSlam = (masc['torneo'] == 'Australian Open') | (masc['torneo'] == 'US Open') | (masc['torneo'] == 'Wimbledon') | (masc['torneo'] == 'French Open')
filtro3sets = (masc['setsA'] == 3) | (masc['setsB'] == 3)
filtro2sets = (masc['setsA'] == 2) | (masc['setsB'] == 2)
filtroGrandSlamGanaA = masc['setsA'] == 3
filtroGrandSlamGanaB = masc['setsB'] == 3
filtroNoGrandSlamGanaA = masc['setsA'] == 2
filtroNoGrandSlamGanaB = masc['setsB'] == 2

# Condición 1: es Grand Slam y no se llegó a 3 sets (partido incompleto/retirado)
cond1 = filtroGrandSlam & (~filtro3sets)
masc.loc[cond1, 'retirado'] = 'si'

# Condición 2: no es Grand Slam y no se llegó a 2 sets (partido incompleto/retirado)
cond2 = (~filtroGrandSlam) & (~filtro2sets)
masc.loc[cond2, 'retirado'] = 'si'

#Creo la variable superficie
torneos_superficies = [
    ('ITF M15 Luan 8 Men', 'hard'),
    ('ITF M15 Monastir 23 Men', 'hard'),
    ('ITF M15 Harmon Men', 'hard'),
    ('ITF M15 Nyiregyhaza Men', 'hard'),
    ('ITF M15 Oradea Men', 'hard'),
    ('ITF M15 Vaasa Men', 'hard'),
    ('ITF M15 Kursumlijska Banja 6 Men', 'hard'),
    ('ITF M15 Kayseri 3 Men', 'hard'),
    ('ITF M25 Kiseljak Men', 'hard'),
    ('ITF M25 Martos Men', 'hard'),
    ('ITF M25 Ceska Lipa Men', 'hard'),
    ('ITF M25 Monastir 2 Men', 'hard'),
    ('ITF M25 Varnamo Men', 'hard'),
    ('ITF M25 Villeneuve-Loubet Men', 'hard'),
    ('ITF M25 Wichita, KS Men', 'hard'),
    ('Lyon Challenger Men', 'clay'),
    ('Bratislava Challenger Men', 'clay'),
    ('Ilkley Challenger Men', 'grass'),
    ('ATP Stuttgart', 'grass'),
    ('ATP Hertogenbosch', 'grass'),
    ('Santa Fe Challenger Men', 'clay'),
    ('Perugia Challenger Men', 'clay'),
    ('ITF M25 Santo Domingo 4 Men', 'hard'),
    ('ITF M15 Messina Men', 'hard'),
    ('ITF M15 San Diego, CA 3 Men', 'hard'),
    ('French Open', 'clay'),
    ('Australian Open', 'hard'),
    ('US Open', 'hard'),
    ('Wimbledon', 'grass')
]

superficie_dict = dict(torneos_superficies)
masc['superficie'] = masc['torneo'].map(superficie_dict)


masc[['dia', 'hora']] = masc['fecha'].str.split(' ', expand=True) #pongo la fecha como de normal (13/01/2025)
masc['dia'] = masc['dia'].str.replace('.', '/', regex=False)
masc['fecha'] = masc['dia'].apply(asignar_año)
masc = masc.drop(columns = ['dia', 'cuotaA', 'cuotaB'])
masc['fecha'] = pd.to_datetime(masc['fecha'], format='%d/%m/%Y')
masc['sets'] = masc['setsA'] + masc['setsB']

#Me quedo con las cuotas buenas
masc = masc.rename(columns={
    'cuotaA_2': 'cuotaA',
    'cuotaB_2': 'cuotaB'
})

#Le quito el símbolo del porcentaje a las columnas que lo tienen
cols_porcentaje = [col for col in masc.columns if 'porcentaje' in col.lower()]
for col in cols_porcentaje:
    masc[col] = masc[col].astype(str).str.replace('%', '', regex=False)
    masc[col] = pd.to_numeric(masc[col], errors='coerce')

#Le quito los paréntesis a las columnas que lo tienen
cols_con_par = [col for col in masc.columns if masc[col].astype(str).str.contains(r'\(\d+/\d+\)').any()]
for col in cols_con_par:
    masc[col] = masc[col].astype(str).str.replace('(', '', regex=False).str.replace(')', '', regex=False)
    
#Creo la columna ganador

masc['ganador'] = -1

ganadorA1 = filtroGrandSlam & filtroGrandSlamGanaA
masc.loc[ganadorA1, 'ganador'] = 0

ganadorB1 = filtroGrandSlam & filtroGrandSlamGanaB
masc.loc[ganadorB1, 'ganador'] = 1

ganadorA2 = (~filtroGrandSlam) & filtroNoGrandSlamGanaA
masc.loc[ganadorA2, 'ganador'] = 0

ganadorB2 = (~filtroGrandSlam) & filtroNoGrandSlamGanaB
masc.loc[ganadorB2, 'ganador'] = 1

itf = masc[masc['torneo'].str.contains("ITF")] #dataset con datos de ITF
challenger = masc[masc['torneo'].str.contains("Challenger")] #dataset con datos de Challenger


masc = masc.sort_values('fecha', ascending=False).reset_index(drop=True)
masc

,torneo,jugadorA,jugadorB,setsA,setsB,fecha,retirado,acesA,acesB,dobleFaltasA,...,porcentajeJuegosGanadosAlRestoA,JuegosGanadosAlRestoA,porcentajeJuegosGanadosAlRestoB,JuegosGanadosAlRestoB,JuegosGanadosA,JuegosGanadosB,superficie,hora,sets,ganador
0,ITF M15 Luan 8 Men,Liu H.,Rawat S.,0,2,2025-06-11,no,NaN,NaN,NaN,...,NaN,nan,NaN,nan,nan,nan,hard,05:35,2,1
1,ITF M15 Monastir 23 Men,Ryan Ziegann S.,Bassem Sobhy M.,2,1,2025-06-11,no,NaN,NaN,NaN,...,NaN,nan,NaN,nan,nan,nan,hard,11:00,3,0
2,ITF M15 Luan 8 Men,Shin W.,Mo Y. C.,0,2,2025-06-11,no,NaN,NaN,NaN,...,NaN,nan,NaN,nan,nan,nan,hard,04:10,2,1
3,ITF M15 Harmon Men,Nakamura R.,Sim S.,2,0,2025-06-11,no,NaN,NaN,NaN,...,NaN,nan,NaN,nan,nan,nan,hard,02:00,2,0
4,ITF M15 Harmon Men,Delaney Ja.,Arcon A.,2,0,2025-06-11,no,NaN,NaN,NaN,...,NaN,nan,NaN,nan,nan,nan,hard,03:30,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899,Wimbledon,Galan D. E.,Albot R.,1,3,2024-06-27,no,18.0,5.0,NaN,...,5.0,1/20,26.0,5/19,15/39,24/39,grass,12:10,4,1
900,Wimbledon,Dzumhur D.,Moro Canas A.,0,2,2024-06-27,si,1.0,4.0,NaN,...,18.0,2/11,25.0,3/12,11/23,12/23,grass,12:10,2,-1
901,Wimbledon,Duckworth J.,Lajal M.,0,3,2024-06-27,no,13.0,12.0,NaN,...,0.0,0/15,20.0,3/15,12/30,18/30,grass,12:10,3,1
902,Wimbledon,Bergs Z.,Yevseyev D.,3,0,2024-06-27,no,11.0,1.0,NaN,...,75.0,9/12,18.0,2/11,18/23,5/23,grass,12:10,3,0


## Limpieza de los datos

Primero vamos a observar el número de nan que tienen las variables

In [152]:
filtro = masc.isna().sum() == 0
masc.isna().sum()[filtro]

torneo                                  0
jugadorA                                0
jugadorB                                0
setsA                                   0
setsB                                   0
fecha                                   0
retirado                                0
juegosGanadosPrimerSaqueA               0
juegosGanadosPrimerSaqueB               0
JuegosGanadosSegundoSaqueA              0
JuegosGanadosSegundoSaqueB              0
BreakPointsSalvadosA                    0
BreakPointsSalvadosB                    0
PuntosGanadosAlRestoConPrimerSaqueA     0
PuntosGanadosAlRestoConPrimerSaqueB     0
PuntosGanadosAlRestoA                   0
PuntosGanadosAlRestoB                   0
PuntosGanadosAlRestoConSegundoSaqueA    0
PuntosGanadosAlRestoConSegundoSaqueB    0
PuntosBreakConvertidosA                 0
PuntosBreakConvertidosB                 0
PuntosGanadosRedA                       0
PuntosGanadosRedB                       0
PuntosGanadosAlSaqueA             

Hay 7 variables sin NaN

In [153]:
filtro = (masc.isna().sum() > 0) & (masc.isna().sum() <= 170)
masc.isna().sum()[filtro]

cuotaA    21
cuotaB    21
dtype: int64

Luego observamos 3 variables con 21 NaN y 1 con 70 NaN. Creé dos variables que extraían las cuotas de manera diferente, veo que la segunda opción es mejor por lo que eliminaré las dos primeras variables.

In [154]:
filtro = masc.isna().sum() == 179
len(masc.isna().sum()[filtro])

34

Hay 58 variables con 179 NaN, lo que probablemente signifique que hay 183 registros incompletos.

In [155]:
filtro = (masc.isna().sum() > 180) & (masc.isna().sum() <= 400)
masc.isna().sum()[filtro]

winnersA                       340
winnersB                       340
erroresNoForzadosA             340
erroresNoForzadosB             340
porcentajePuntosGanadosRedA    341
porcentajePuntosGanadosRedB    341
dtype: int64

Hay 8 variables con 340 y 341 NaN.

In [156]:
filtro = (masc.isna().sum() > 400) & (masc.isna().sum() <= 900)
masc.isna().sum()[filtro]
columnas_a_eliminar = masc.isna().sum()[filtro].index.tolist()
print(columnas_a_eliminar)
masc = masc.drop(columns = columnas_a_eliminar)

['velocidadMediaPrimerSaqueA', 'velocidadMediaPrimerSaqueB', 'velocidadMediaSegundoSaqueA', 'velocidadMediaSegundoSaqueB']


Hay 4 variables con 762 NaN. Las elimino.

In [157]:
filtro = masc.isna().sum() > 900
print(len(masc.isna().sum()[filtro]))
columnas_a_eliminar = masc.isna().sum()[filtro].index.tolist()
masc = masc.drop(columns = columnas_a_eliminar)
print(columnas_a_eliminar[:5])

46
['dobleFaltasA', 'dobleFaltasB', 'porcentajePuntosGanadosConPrimerSaqueA', 'porcentajePuntosGanadosConPrimerSaqueB', 'puntosGanadosConPrimerSaqueA']


Por último hay 46 variables con todo NaN, significa que no he extraído bien los datos de estas variables. También las elimino.

In [158]:
masc.shape

(904, 79)

Nos quedamos con 77 variables.

In [159]:
print(len(itf))
print(len(masc[masc['dobleB'].isna()]))

179
179


Ejemplo

In [160]:
jugador = "Alcaraz C."

df = df_jugador(jugador, masc)

display(df.head())

print(df.columns.tolist())

,torneo,jugadorA,jugadorB,setsA,setsB,fecha,retirado,acesA,acesB,porcentajePrimerSaqueA,...,porcentajeJuegosGanadosAlRestoA,JuegosGanadosAlRestoA,porcentajeJuegosGanadosAlRestoB,JuegosGanadosAlRestoB,JuegosGanadosA,JuegosGanadosB,superficie,hora,sets,ganador
8,French Open,Alcaraz C.,Sinner J.,3,2,2025-06-08,no,7.0,8.0,58.0,...,25.0,7/28,25.0,7/28,28/56,28/56,clay,15:30,5,1
9,French Open,Alcaraz C.,Musetti L.,2,1,2025-06-06,si,1.0,2.0,58.0,...,40.0,6/15,20.0,3/15,18/30,12/30,clay,14:40,3,-1
10,French Open,Alcaraz C.,Paul T.,3,0,2025-06-03,no,3.0,3.0,78.0,...,55.0,6/11,0.0,0/12,18/23,5/23,clay,20:30,3,1
11,French Open,Alcaraz C.,Shelton B.,3,1,2025-06-01,no,5.0,6.0,70.0,...,15.0,3/20,10.0,2/21,22/41,19/41,clay,16:50,4,1
12,French Open,Alcaraz C.,Dzumhur D.,3,1,2025-05-30,no,4.0,0.0,70.0,...,39.0,7/18,17.0,3/18,22/36,14/36,clay,20:30,4,1


['torneo', 'jugadorA', 'jugadorB', 'setsA', 'setsB', 'fecha', 'retirado', 'acesA', 'acesB', 'porcentajePrimerSaqueA', 'porcentajePrimerSaqueB', 'porcentajeBreakPointsSalvadosA', 'porcentajeBreakPointsSalvadosB', 'porcentajePuntosGanadosAlRestoConPrimerSaqueA', 'porcentajePuntosGanadosAlRestoConPrimerSaqueB', 'porcentajePuntosGanadosAlRestoConSegundoSaqueA', 'porcentajePuntosGanadosAlRestoConSegundoSaqueB', 'winnersA', 'winnersB', 'erroresNoForzadosA', 'erroresNoForzadosB', 'porcentajePuntosGanadosRedA', 'porcentajePuntosGanadosRedB', 'porcentajePuntosGanadosA', 'porcentajePuntosGanadosB', 'porcentajeJuegosGanadosA', 'porcentajeJuegosGanadosB', 'cuotaA', 'cuotaB', 'dobleA', 'dobleB', 'porcentajeJuegosGanadosPrimerSaqueA', 'juegosGanadosPrimerSaqueA', 'porcentajeJuegosGanadosPrimerSaqueB', 'juegosGanadosPrimerSaqueB', 'porcentajeJuegosGanadosSegundoSaqueA', 'JuegosGanadosSegundoSaqueA', 'porcentajeJuegosGanadosSegundoSaqueB', 'JuegosGanadosSegundoSaqueB', 'BreakPointsSalvadosA', 'BreakPo

In [161]:
acesRealizadosHierba = float(df[df['superficie'] == 'grass']['acesA'].mean())
acesRecibidosHierba = float(df[df['superficie'] == 'grass']['acesB'].mean())
doblesRealizadasHierba = float(df[df['superficie'] == 'grass']['dobleA'].mean())

acesRealizadosDura = float(df[df['superficie'] == 'hard']['acesA'].mean())
acesRecibidosDura = float(df[df['superficie'] == 'hard']['acesB'].mean())
doblesRealizadasDura = float(df[df['superficie'] == 'hard']['dobleA'].mean())

acesRealizadosTierra = float(df[df['superficie'] == 'clay']['acesA'].mean())
acesRecibidosTierra = float(df[df['superficie'] == 'clay']['acesB'].mean())
doblesRealizadasTierra = float(df[df['superficie'] == 'clay']['dobleA'].mean())

print("Aces realizados en hierba:", acesRealizadosHierba)
print("Aces recibidos en hierba:", acesRecibidosHierba)
print("Dobles faltas realizadas en hierba:", doblesRealizadasHierba)
print("Aces realizados en dura:", acesRealizadosDura)
print("Aces recibidos en dura:", acesRecibidosDura)
print("Dobles faltas realizadas en dura:", doblesRealizadasDura)
print("Aces realizados en tierra:", acesRealizadosTierra)
print("Aces recibidos en tierra:", acesRecibidosTierra)
print("Dobles faltas realizadas en tierra:", doblesRealizadasTierra)

Aces realizados en hierba: 9.0
Aces recibidos en hierba: 6.142857142857143
Dobles faltas realizadas en hierba: 3.857142857142857
Aces realizados en dura: 7.857142857142857
Aces recibidos en dura: 2.5714285714285716
Dobles faltas realizadas en dura: 3.5714285714285716
Aces realizados en tierra: 4.0
Aces recibidos en tierra: 4.142857142857143
Dobles faltas realizadas en tierra: 3.0
